In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [2]:
# code adapted from ChatGPT

# Define your neural network model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(2, 1, bias=False)  # Input size 2, output size 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        # RUN IT ON A GPU if it exists
        if torch.cuda.is_available():
            x = x.to("cuda")

        x = self.fc(x)
        x = self.sigmoid(x)
        
        return x

def train_model(inputs, desired_outputs, num_epochs=100, learning_rate=0.02, momentum=0.9):
    # Convert inputs and desired_outputs to PyTorch tensors
    inputs = torch.tensor(inputs, dtype=torch.float32)
    desired_outputs = torch.tensor(desired_outputs, dtype=torch.float32)
    
    # Create a DataLoader to handle batching (if needed)
    dataset = TensorDataset(inputs, desired_outputs)
    dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)  # Adjust batch_size as needed
    
    # Initialize the model
    model = SimpleModel()
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()  # Mean Squared Error loss
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    
    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_inputs, batch_desired_outputs in dataloader:
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_inputs)  # Forward pass
            loss = criterion(outputs, batch_desired_outputs)  # Compute the loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update the model's parameters
            cur_item = loss.item()
            total_loss += cur_item
            # print(cur_item)
        
        # Print the average loss for this epoch
        print(f"total loss: {total_loss}")
        average_loss = total_loss / len(dataloader)
        for i, param in model.named_parameters():
            print(param)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')
    
    # Return the trained model
    return model

In [3]:
input_data = np.random.randint(1, 100, size=(1000, 2))
desired_output = []

for i in input_data:
    desired_output.append([int(i[0] >= i[1])])
    
# Train the model
trained_model = train_model(input_data, desired_output, 500)


total loss: 0.06897056102752686
Parameter containing:
tensor([[ 0.5231, -0.4794]], requires_grad=True)
Epoch [1/500], Loss: 0.0690
total loss: 0.029300354421138763
Parameter containing:
tensor([[ 0.4892, -0.5173]], requires_grad=True)
Epoch [2/500], Loss: 0.0293
total loss: 0.019920913502573967
Parameter containing:
tensor([[ 0.4731, -0.5376]], requires_grad=True)
Epoch [3/500], Loss: 0.0199
total loss: 0.048463501036167145
Parameter containing:
tensor([[ 0.4747, -0.5413]], requires_grad=True)
Epoch [4/500], Loss: 0.0485
total loss: 0.04978193715214729
Parameter containing:
tensor([[ 0.4921, -0.5302]], requires_grad=True)
Epoch [5/500], Loss: 0.0498
total loss: 0.026991553604602814
Parameter containing:
tensor([[ 0.5236, -0.5053]], requires_grad=True)
Epoch [6/500], Loss: 0.0270
total loss: 0.010762839578092098
Parameter containing:
tensor([[ 0.5427, -0.4926]], requires_grad=True)
Epoch [7/500], Loss: 0.0108
total loss: 0.033781908452510834
Parameter containing:
tensor([[ 0.5429, -0.49

In [45]:
def test_model(model, new_data, expected_result):
    test_input = torch.tensor(new_data, dtype=torch.float32)
    test_output = trained_model(test_input)

    # apply the step function with the step at 0.5 to decide whether to predict 1 or 0 for this element
    predicted_classes = torch.tensor(test_output + 0.5, dtype=torch.int)
    test_output_compare = torch.tensor(expected_result, dtype=torch.float32)

    diff = abs(predicted_classes - test_output_compare)
    accuracy = 1 - (int(sum(diff)) / len(test_input))
    return accuracy

In [38]:
test_input = np.random.randint(1, 100, size=(500, 2))
test_output_compare = []

for i in test_input:
    test_output_compare.append([int(i[0] >= i[1])])

test_model(trained_model, test_input, test_output_compare)

In [47]:
for i, param in trained_model.named_parameters():
    print(i)
    print(param)

fc.weight
Parameter containing:
tensor([[ 0.9054, -0.9015]], requires_grad=True)
